## CHALLENGE: model evaluation

We got the houseprice data with set of variables. The task is to create a regression model that will give the best performance measurements. We will try to use different types of regression models and do cross validation. 
Let's take a look at the data:

In [103]:
import numpy as np
import pandas as pd
from sklearn import linear_model
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import statsmodels.api as sm
from statsmodels.tools.eval_measures import mse, rmse
import seaborn as sns
import warnings
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV, ElasticNetCV

warnings.filterwarnings('ignore')

postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'houseprices'

In [104]:
engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))
house_df = pd.read_sql_query('select * from houseprices',con=engine)

# no need for an open connection, as we're only doing a single query
engine.dispose()
house_df ['Year'] = house_df ['yrsold']
house_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 82 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             1460 non-null   int64  
 1   mssubclass     1460 non-null   int64  
 2   mszoning       1460 non-null   object 
 3   lotfrontage    1201 non-null   float64
 4   lotarea        1460 non-null   int64  
 5   street         1460 non-null   object 
 6   alley          91 non-null     object 
 7   lotshape       1460 non-null   object 
 8   landcontour    1460 non-null   object 
 9   utilities      1460 non-null   object 
 10  lotconfig      1460 non-null   object 
 11  landslope      1460 non-null   object 
 12  neighborhood   1460 non-null   object 
 13  condition1     1460 non-null   object 
 14  condition2     1460 non-null   object 
 15  bldgtype       1460 non-null   object 
 16  housestyle     1460 non-null   object 
 17  overallqual    1460 non-null   int64  
 18  overallc

In [76]:
#Let's add investment flow data
Investment = pd.read_csv ('monthly_csv_Investm.csv')
Investment['Year'] = Investment['Date'].str[:4]
Investment['Year'] = Investment['Year'].astype(int)
Investment.head()

,Date,Total Equity,Domestic Equity,World Equity,Hybrid,Total Bond,Taxable Bond,Municipal Bond,Total,Year
0,2007-01-31,27364,5723,21641,5321,15287,12453,2834,47972,2007
1,2007-02-28,25306,8411,16895,5164,15064,11926,3137,45533,2007
2,2007-03-31,6551,-486,7037,3764,15782,12925,2857,26097,2007
3,2007-04-30,16063,-163,16225,4384,13701,12346,1355,34148,2007
4,2007-05-31,-2876,-14176,11300,4318,20813,17215,3598,22256,2007


In [80]:
Investment_df = Investment.groupby(["Year"])["Total Bond"].sum().head()

In [81]:
#Let's add unemployment rate and 
Unemployment = pd.read_csv ('aat1_csv.csv')
Unemployment['Year'] = Unemployment['year']
Unemployment['Year'] = Unemployment['Year'].astype(int)
Unemployment.head(100)

,year,population,labor_force,population_percent,employed_total,employed_percent,agrictulture_ratio,nonagriculture_ratio,unemployed,unemployed_percent,not_in_labor,footnotes,Year
0,1941,99900,55910,56.0,50350,50.4,9100,41250,5560,9.9,43990,NaN,1941
1,1942,98640,56410,57.2,53750,54.5,9250,44500,2660,4.7,42230,NaN,1942
2,1943,94640,55540,58.7,54470,57.6,9080,45390,1070,1.9,39100,NaN,1943
3,1944,93220,54630,58.6,53960,57.9,8950,45010,670,1.2,38590,NaN,1944
4,1945,94090,53860,57.2,52820,56.1,8580,44240,1040,1.9,40230,NaN,1945
...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,2006,228815,151428,66.2,144427,63.1,2206,142221,7001,4.6,77387,1.0,2006
67,2007,231867,153124,66.0,146047,63.0,2095,143952,7078,4.6,78743,1.0,2007
68,2008,233788,154287,66.0,145362,62.2,2168,143194,8924,5.8,79501,1.0,2008
69,2009,235801,154142,65.4,139877,59.3,2103,137775,14265,9.3,81659,1.0,2009


In [82]:
result = pd.merge (Investment_df, Unemployment, on  = 'Year')
result.head(10)

,Year,Total Bond,year,population,labor_force,population_percent,employed_total,employed_percent,agrictulture_ratio,nonagriculture_ratio,unemployed,unemployed_percent,not_in_labor,footnotes
0,2007,110609,2007,231867,153124,66.0,146047,63.0,2095,143952,7078,4.6,78743,1.0
1,2008,30032,2008,233788,154287,66.0,145362,62.2,2168,143194,8924,5.8,79501,1.0
2,2009,371238,2009,235801,154142,65.4,139877,59.3,2103,137775,14265,9.3,81659,1.0
3,2010,232342,2010,237830,153889,64.7,139064,58.5,2206,136858,14825,9.6,83941,1.0


In [96]:
result_df = pd.merge (house_df, result, on  = 'Year')
result_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1146 entries, 0 to 1145
Data columns (total 95 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    1146 non-null   int64  
 1   mssubclass            1146 non-null   int64  
 2   mszoning              1146 non-null   object 
 3   lotfrontage           936 non-null    float64
 4   lotarea               1146 non-null   int64  
 5   street                1146 non-null   object 
 6   alley                 72 non-null     object 
 7   lotshape              1146 non-null   object 
 8   landcontour           1146 non-null   object 
 9   utilities             1146 non-null   object 
 10  lotconfig             1146 non-null   object 
 11  landslope             1146 non-null   object 
 12  neighborhood          1146 non-null   object 
 13  condition1            1146 non-null   object 
 14  condition2            1146 non-null   object 
 15  bldgtype             

In [86]:
result_df = pd.concat([house_df,pd.get_dummies(house_df.exterqual, prefix="exter", drop_first=True)], axis=1)
result_df = pd.concat([house_df,pd.get_dummies(house_df.kitchenqual, prefix="kitchen", drop_first=True)], axis=1)
dummy_column_names = list(pd.get_dummies(result_df.exterqual, prefix="exter", drop_first=True).columns)
dummy_column_names = dummy_column_names + list(pd.get_dummies(result_df.kitchenqual, prefix="kitchen", drop_first=True).columns)

In [101]:
#Let's add to the model two more variables  - unemployed_percent and total bond
#Now let's exclude statistically insignificant variables from the model and re-estimate it:
Y = result_df['saleprice']
# X is the feature set
X = result_df[['overallqual',
              'grlivarea', 'garagearea', 
              'totalbsmtsf', 'firstflrsf',
              'yearbuilt', 'yearremodadd', 'unemployed_percent', 'Total Bond']]
X = sm.add_constant(X)

results = sm.OLS(Y, X).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.762
Model:                            OLS   Adj. R-squared:                  0.761
Method:                 Least Squares   F-statistic:                     405.1
Date:                Tue, 04 Feb 2020   Prob (F-statistic):               0.00
Time:                        05:27:24   Log-Likelihood:                -13733.
No. Observations:                1146   AIC:                         2.749e+04
Df Residuals:                    1136   BIC:                         2.754e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const              -1.313e+06   1.41e+05     -9.339      0.000   -1.59e+06   -1.04e+06
overallqual         1.947e+04   1355.420     14.365      0.000    1.68e+04    2.21e+04
grlivarea             46.2031      3.072     15.040      0.000      40.176      52.230
garagearea            42.3237      7.065      5.990      0.000      28.461      56.186
totalbsmtsf           20.3534      4.766      4.271      0.000      11.002      29.704
firstflrsf            11.7919      5.605      2.104      0.036       0.794      22.790
yearbuilt            278.8767     53.521      5.211      0.000     173.865     383.888
yearremodadd         352.6656     71.196      4.953      0.000     212.975     492.356
unemployed_percent     1.3855    985.474      0.001      0.999   -1932.169    1934.940
Total Bond            -0.0011      0.016     -0.071      0.943      -0.032       0.029
==============================================================================
Omnibus:                      419.312   Durbin-Watson:                   2.125
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            58564.212
Skew:                          -0.577   Prob(JB):                         0.00
Kurtosis:                      38.002   Cond. No.                     2.81e+07
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.81e+07. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [99]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1146 entries, 0 to 1145
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   const               1146 non-null   float64
 1   overallqual         1146 non-null   int64  
 2   grlivarea           1146 non-null   int64  
 3   garagearea          1146 non-null   int64  
 4   totalbsmtsf         1146 non-null   int64  
 5   firstflrsf          1146 non-null   int64  
 6   yearbuilt           1146 non-null   int64  
 7   yearremodadd        1146 non-null   int64  
 8   unemployed_percent  1146 non-null   float64
dtypes: float64(2), int64(7)
memory usage: 89.5 KB


In [105]:
from sklearn import ensemble
from sklearn.model_selection import cross_val_score

rfc = ensemble.RandomForestClassifier()

cross_val_score(rfc, X, Y, cv=10)

array([0.0104712 , 0.01809955, 0.0075188 , 0.        , 0.02040816,
       0.06060606, 0.        , 0.06666667, 0.        , 0.        ])